In [1]:
!pip install ultralytics opencv-python-headless

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 330.3 kB/s eta 0:01:58
   ---------------------------------------- 0.0/38.8 MB 435.7 kB/s eta 0:01:29
   ---------------------------------------- 0.0/38.8 MB 281.8 kB/s eta 0:02:18
   ---------------------------------------- 0.1/38.8 MB 328.2 kB/s eta 0:01:58
   ---------------------------------------- 0.1/38.8 MB 420.8 kB/s eta 0:01:32
   ---------------------------------------- 0.2/38.8 MB 541.0 kB/s eta 0:01:12
   ---------------------------------------- 0.2/38.8 MB 655.1 kB/s eta 0:00:59
   ---------------------------------------- 0.2/38.8 MB 686.8 kB/s eta 0:00:57
   ---------------------------------------- 0.2/38.8 MB 686.8 kB/s eta 0:00:57
   ---------------------------------------- 0.2/38.8 MB 686.8 kB/s eta 0:00:57
   ---------------------------------------- 0.2/38.8 MB 686.8 kB/s eta 0:00:57
   ---------------------------------------- 0.2/38.8 MB 686.8

In [3]:
import os
import shutil
import random
from pathlib import Path

# Define paths
train_images_path = Path(r"C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\images")
train_labels_path = Path(r"C:\Users\Mohammad\Desktop\New folder (2)\archive-6\train-pure\train-pure\labels")
valid_images_path = Path(r"C:\Users\Mohammad\Desktop\New folder (2)\archive-6\valid\valid\images")
valid_labels_path = Path(r"C:\Users\Mohammad\Desktop\New folder (2)\archive-6\valid\valid\labels")

# Create directories for train, test and valid split
base_path = Path(r"C:\Users\Mohammad\Desktop\YOLOv8_dataset")
base_path.mkdir(parents=True, exist_ok=True)

for split in ['train', 'test', 'valid']:
    (base_path / split / 'images').mkdir(parents=True, exist_ok=True)
    (base_path / split / 'labels').mkdir(parents=True, exist_ok=True)

# Function to split dataset
def split_dataset(train_images_path, train_labels_path, test_ratio=0.25):
    image_files = list(train_images_path.glob('*.jpg'))
    random.shuffle(image_files)
    
    test_size = int(len(image_files) * test_ratio)
    test_files = image_files[:test_size]
    train_files = image_files[test_size:]
    
    return train_files, test_files

train_files, test_files = split_dataset(train_images_path, train_labels_path)

# Copy train files
for file in train_files:
    shutil.copy(file, base_path / 'train' / 'images' / file.name)
    shutil.copy(train_labels_path / (file.stem + '.txt'), base_path / 'train' / 'labels' / (file.stem + '.txt'))

# Copy test files
for file in test_files:
    shutil.copy(file, base_path / 'test' / 'images' / file.name)
    shutil.copy(train_labels_path / (file.stem + '.txt'), base_path / 'test' / 'labels' / (file.stem + '.txt'))

# Copy valid files
for file in valid_images_path.glob('*.jpg'):
    shutil.copy(file, base_path / 'valid' / 'images' / file.name)
    shutil.copy(valid_labels_path / (file.stem + '.txt'), base_path / 'valid' / 'labels' / (file.stem + '.txt'))


In [7]:
import os
import shutil
import random
from pathlib import Path
# Create directories for train, test and valid split
base_path = Path(r"C:\Users\Mohammad\Desktop\YOLOv8_dataset")
base_path.mkdir(parents=True, exist_ok=True)

data_config = """
path: {}
train: train/images
val: valid/images
test: test/images

nc: 1  # Number of classes
names: ['pavement']
""".format(base_path)

config_path = base_path / 'data.yaml'
with open(config_path, 'w') as f:
    f.write(data_config)


In [9]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.yaml')

# Train the model
model.train(data=str(config_path), epochs=50, imgsz=640, batch=8, device=0, name='yolov8_road_pavement')



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002362EA5DA30>
fitness: 0.21729174670167062
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.19092])
names: {0: 'pavement'}
plot: True
results_dict: {'metrics/precision(B)': 0.5204478350845188, 'metrics/recall(B)': 0.46400740769463866, 'metrics/mAP50(B)': 0.45466901080063477, 'metrics/mAP50-95(B)': 0.19091649513511902, 'fitness': 0.21729174670167062}
save_dir: WindowsPath('runs/detect/yolov8_road_pavement2')
speed: {'preprocess': 0.22117661827604204, 'inference': 2.2050540564938883, 'loss': 0.0, 'postprocess': 1.4340833736205074}

In [17]:
# Save the model
# import torch
# num_initial_epochs = 50
# torch.save({
#     'epoch': num_initial_epochs,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict()
# }, 'yolov8_checkpoint.pth')

model_path = 'C:/Users/Mohammad/Desktop/New folder (2)/archive-6/yolov8_road_pavement.pt'
model.save(model_path)
print(f"Model saved to {model_path}")

AttributeError: 'YOLO' object has no attribute 'save'

In [ ]:
loaded_model = YOLO(model_path)
print(f"Model loaded from {model_path}")

In [33]:
# Evaluate on validation set
# metrics = model.val(data=str(config_path))

# Predict on a test image
# test_image_path = base_path / 'test' / 'images'
results = model.val(data=str(config_path), split="test")
results.show()


Ultralytics YOLOv8.0.196  Python-3.12.4 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Scanning C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\labels... 1589 images, 0 backgrounds, 0 corrupt: 100%|█████
val: WARNING  C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\images\G0011614_jpg.rf.ccad2055aa6ecc87078ca39ff30513d0.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\images\G0011661_jpg.rf.f81cbd213dfad9a82cc707688a8377ba.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\images\G0011664_jpg.rf.dbe6c8390b11d7fb7f763f1b106885bd.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\images\G0011773_jpg.rf.f464fe4397fa996d8223bf9e91438353.jpg: 2 duplicate labels removed
val: WARNING  C:\Users\Mohammad\Desktop\YOLOv8_dataset\test\images\G0011978_jpg.rf.8de8cd01d2e23a82553f72e30a05270b.jpg: 2 duplicate labels removed
val: WARNING  C:\Users\Mohamm

AttributeError: 'DetMetrics' object has no attribute 'show'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots pat

In [ ]:
results.